# FLIP (00): Data Science 
**(Module 03: Linear Algebra)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use,but NOT allowed to change and distribute this package.

Prepared by and for 
**Student Members** |
2006-2022 [TULIP Lab](http://www.tulip.org.au), Australia

---
## Session 23 Logistic Regression

##  <span style="color:black;"> Logit function:</span>
A logit function is simply a function of the mean of the response variable Y that we use as the response instead of Y itself.

Because Y is a categorical binary variable and we need to predict in percentages, the logistic model uses the logit function to help us transform it into a response between 0 and 1

In the formula below: $\beta_{0} + \beta_{1}X_{1} $ for simple logisitic regression can be changed for multiple logisitic regression for the linear compination of independent variables $\beta_{0} + \beta_{1}X_{1} + \beta_{2}X_{2} + \dotso + \beta_{k}X_{k } $ They signify the linear compination of the independent variables.



 $ Ln  \bigg( \dfrac{{P}} {1 - P} \bigg) = \beta_{0} + \beta_{1}X_{1}  $ 


### <span style="color:black;">Visualizing the logit:</span>
> $ logit(x) = Ln  \bigg( \dfrac{{x}} {1 - x} \bigg)  $

In [ ]:
import numpy as np
from pylab import *
p_x = np.arange(0.0001,0.999999999,0.0001).tolist()

r_y = [np.log(i/(1-i)) for i in p_x]

def set_data_source(r_x,r_y):
    return ColumnDataSource(
        data=dict(
            x=r_x, 
            y=r_y,
        )
    )
    
def plot(l_x = [], l_y = [], s_x = [], s_y = [],
         width = 600,height = 400, 
         x_label = "", y_label = "", title = "", 
         line = False, scatter = False):

    plot = figure(plot_width=width, plot_height=height, 
              x_axis_label = x_label,
                y_axis_label = y_label,
              title=title,
            )
    if line:
        l_source = set_data_source(l_x,l_y)
        plot.line('x','y', source = l_source)
        
    if scatter:
        s_source = set_data_source(s_x,s_y)
        plot.circle('x', 'y', size=15, source = s_source)
        
    show(plot)

plot(p_x,r_y,[],[],400,400,"Percentages","", "Logit function: Ln(P/1-P)", True)


You can see it is reaching to -inf and inf as we get close to 0 and 1

Then we compute the inverse logit because we want our percentage as the Y variable / dependent. 

The inverse log of the the logit function allows us to find estimated regression equation.

$ \dfrac{{P}} {1 - P} = e^{\beta_{0} + \beta_{1}X_{1}}  $ 

$ P = e^{\beta_{0} + \beta_{1}X_{1} } \big( {1 - P} \big) $ 


... Until we end up with the estimated regression equation

$ \widehat{P} = \dfrac{e^{\beta_{0} + \beta_{1}X_{1}}} {1 + e^{\beta_{0} + \beta_{1}X_{1}} } $



### <span style="color:black;">Visualizing the inverse logit:</span>
> $ logit^{-1}(a) = \dfrac{e^{a} } {1 + e^{a}}  $

> We can see below that the function forms a sigmoid curve or S curve



In [ ]:
p_x = np.arange(-10,10,0.001).tolist()

r_y = [np.exp(i)/(1+np.exp(i)) for i in p_x]

plot(p_x,r_y,[],[],600,400,"","Percentages","Inverse Logit function: e(a)/1-e(a)", True)

## <span style="color:black;"> A simple example calculated </span>

Lets say we are an individual applying for a loan in an Icelandic bank and
want to know the probability of our creditscore getting us a loan at the bank.

We have our individual credit scores and their categorical binary value.
##### Data for each applicant:
* x-axis: Icelandic credit score points ranging 100(bad credit score) to 400(excellent)
* y-axis: Approved as 1 or 0

You can get your individual credit score from [Credit Info](https://www.creditinfo.is)

In [ ]:
# Read the data in
from pandas import *
data = pd.read_csv("F:\Recourse\Logistic-Regression-master\Logistic-Regression-master\loans.csv")

df = data.loc[:,('accepted','creditscore')]
x_credit_scores = df.creditscore.values.tolist()
y_zeroes_and_ones = df.accepted.values.tolist()

sample_size = sum(df.accepted.value_counts())

plot([],[],x_credit_scores,y_zeroes_and_ones, 800, 400, 
     "Credit score points","Accepted","Credit Scores of " + str(sample_size) + " individuals on a scatter plot",
    False, True)

print("Our data has", sample_size, "rows")
print(df.accepted.value_counts())


Next we want to plot the line but first we need to find out our coefficients.

The regression coefficients represent the change in the logit for each unit change in the predictor(i.e.indepenent variable)

The regression coefficients for logistic regression are calculated using maximum likelihood estimation or MLE. That is a topic of machine learning algorithms and a whole other lecture. If you want more info on MLE from
statsmodel you can call print(result.mle_settings). Our example reveales that MLE uses the Newton optimizer.


We can use the Logit function in statsmodel to get the coefficients for us.

Lets look at the data:

In [ ]:
print(df.head(), '\n')
print (df.describe(), '\n')
# print ("Std deviation:",df.std(), sep='\n')
print ("Std deviation:",df.std())

Count accepted vs. not accepted

In [ ]:
print(df.accepted.value_counts())

### Histograms

In [ ]:
# Make histograms from the data
df.hist()

pl.show()

### Finding the slope (b) and intercept (b0) of the best-fitting equation
* We will be using the maximum-likelihood method through the Logit method in statsmodel

> ##### According to the statsmodel docs on the Logit method:
    An intercept is not included by default so it should be added by the user.

In [ ]:
df['intercept'] = 1.00

train_cols = df.loc[:,('creditscore','intercept')]
# Index(['creditscore', 'intercept'], dtype='object')
#dfm = dfm.loc[:,('accepted','creditscore','amount','age','marital','health_ins')]

# the Logit uses the Newton optimizer in Maximum Likelihood Estimation (MLE), 
# (see print(result.mle_settings) )

logit = sm.Logit(df['accepted'], train_cols)

result = logit.fit()
coef_arr = result.params.values
#credit_coef, intercept_coef = np.split(coef_arr,2)
#print(result.resid_generalized)
print(result.summary())
#print( result.conf_int())
#print( np.exp(result.params))

**From our output we get a <u>positive</u> coefficient slope for the credit score.**

**That means that the probability of getting a loan <u>increases</u> as we get a higher credit score.**

If we have P>|z| value of 0.000 which is higher then 0.001 and then that indipendent variable is significant.

 ##### Now we can put the coefficients into the estimated regression equation:
$$ \widehat{P} = \dfrac{e^{\beta_{0} + \beta_{1}X_{1}}} {1 + e^{\beta_{0} + \beta_{1}X_{1}} } $$

 $$ X_{1} \text{is the value we want to test}$$

In [ ]:
# Input we want to test
test_x = [230]

def phat(coef_arr,test_x_arr):
    coef_arr, intercept = coef_arr[:-1],coef_arr[-1]
    slopes = sum(list(c * x for c,x in zip(coef_arr,test_x_arr)))
    ''' # Testings; use if you want to test
    slopes = 0
    for c,x in zip(coef_arr, test_x_arr):
        print(c,"*",x)
        slopes = slopes + c * x
        print("slopes:",slopes)'''
    
    a = intercept + slopes
    return np.exp(a) / (1 + np.exp(a))

p_hat = phat(coef_arr,test_x)
print(p_hat,"\n")
print("Someone with a credit score of", test_x, "has an estimated probabilty of", "%0.2f" % (p_hat*100), "% to get a loan." )


##### Odds

In [ ]:
odds = p_hat/(1-p_hat)
print("odds for ",test_x, "are:", odds, "to 1 or", Fraction(odds),"to 1")

'''Then lets take someone with 231'''
test2_x = [231]
p_hat2 = phat(coef_arr,test2_x)
odds2 = p_hat2/(1-p_hat2)
print("odds for ",test2_x, "are:", odds2, "to 1 or", Fraction(odds2),"to 1")

odds_ratio =  odds2 / odds
print("odds ratio is", odds_ratio)
print("Another way of calculating the odds ratio is from the coefficients")
print("By putting our coefficients to the power of e or",np.exp(1),"we get the odds ratio:")
print((np.exp(result.params)))

print("So the odds of ", odds_ratio,"holds true for any 1 unit increase for any credit score.")
print("If you had a score of 380 and increased to 381 it would have the same odds ratio.")

In [ ]:
line_x = [i for i in range(100,400)]
line_y = [phat(coef_arr,[x]) for x in range(100,400)]
plot(line_x,line_y, x_credit_scores,y_zeroes_and_ones,800, 400, 
     "Credit score","Percentages","Credit Scores of " + str(sample_size) + " individuals on a scatter plot",
    True, True)

## <span style="color:black">Multiple Logistic Regression:</span>


* What is the probability for Olaf getting a loan with:
    * credit_score = 250.
    * loan_amount = 20000000
    * age = 80
    * marital = 0
    * health_insurance = 0
    
We have to:
1. Read the loans.csv file
2. Only get the columns we will be using. 
    * we can use df.loc[:,('col1','col2','col3')] or slicing into another variable.
3. Calculate the coefficients
4. Are there any risk factors for not getting a loan ?
5. Calculate the probability of Olaf getting a loan


In [ ]:
# Read the data in
dfm = pd.read_csv("F:\Recourse\Logistic-Regression-master\Logistic-Regression-master\loans.csv")


# All the columns
'''accepted,creditscore,amount,age,marital,health_ins,creditgrade'''
# Get only columns we want
dfm = dfm.loc[:,('accepted','creditscore','amount','age','marital','health_ins')]
'''accepted,creditscore,amount,age,marital,health_ins'''

print(dfm.head(), '\n')
print (dfm.describe(), '\n')
print ("Std deviation:",dfm.std())

### Frequency tables and histograms

In [ ]:
# Frequency tables for our binary data 
# print(pd.crosstab(dfm['accepted'],dfm['age'], rownames=['accepted or not']))
print(pd.crosstab(dfm['accepted'],dfm['marital'],rownames=['accepted or not']))
print(pd.crosstab(dfm['accepted'],dfm['health_ins'], rownames=['accepted or not']))

# Make histograms from the data
dfm.hist()
pl.show()

In [ ]:
dfm['intercept'] = 1.00

m_train_cols = dfm.columns[1:]
print(m_train_cols)

# the Logit uses the Newton optimizer in Maximum Likelihood Estimation (MLE), 
# (see print(result.mle_settings) )

m_logit = sm.Logit(dfm['accepted'], dfm[m_train_cols])

result = m_logit.fit()
coef_arr_2 = result.params.values
#print(result.resid_generalized)
print(result.summary())

In [ ]:

'''Olafs info
credit_score = 250,
loan_amount = 20000000,
age = 80,
marital = 0,
health_insurance = 0,
'''
olafs_info = [250,20000000,80,0,0]

olaf_phat = phat(coef_arr_2,olafs_info)
print("With a credit score of:", olafs_info[0])
print("loan amount of:", olafs_info[1])
print("age:",olafs_info[2])
print("marital status:",olafs_info[3])
print("and health insurance:",olafs_info[4])
print("Olaf has an estimated probabilty of", "%0.2f" % (olaf_phat*100), "% to get a loan." )


If the probability of getting a loan is p then not getting a loan would be q = 1 - p

**To sum it up the LR model is all about solving for p based on the values of the independent variables**

In [ ]:
print("Olaf has an estimated probabilty of", "%0.2f" % ((1-olaf_phat)*100), "% of NOT getting a loan.")

#### <span style="color:black;"> Appendix 1</span>
### <span style="color:black;">Generation code for loan data:</span>




In [ ]:
#!/usr/bin/env python3
import sys
import os
from random import randint
from math import ceil
from datetime import datetime

# Config Variables
'''Remove the loans.csv file'''
# % rm loans.csv # Comment this line if you are not running this in jupyter notebook
               # or know what you are doing. Currently this does not support partial data
                # we have a read_file function that needs work and other components need some work


loan_info_file = "F:\Recourse\Logistic-Regression-master\Logistic-Regression-master\loans.csv"
if os.path.exists(loan_info_file):
    os.remove(loan_info_file)
    print("removed old", loan_info_file)

lines_for_each_part = 3000
#------ 
# Constants
''' parts is constant 5 because credit score grades are in 5 parts'''
parts = 5


def is_not_zero_file(fpath):
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

def read_file(ppl):
    with open(loan_info_file, 'r') as f:
        for l in f.read().split('\n')[1:]:
            l = l.strip()
            if not l:
                continue
            l = l.split(',')
            per = {}
            per['accepted'] = l[0]
            if len(l) > 1:
                per['creditscore'] = l[1]
                per['amount'] = l[2]
                per['age'] = l[3]
                per['marital'] = l[4]
            
            ppl.append(per)

def create_new(ppl):

    # Try to generate evenly distributed 0 and 1 for accepted/not accepted:
    '''
    To help us do that we will be using
    The Credit Info credit grade that are mapped unto the points defined as follows
    A = 353-400+, 
    B = 317-353, 
    C = 281-317, 
    D = 235-281, 
    E = 100-235

    E and D applicants do not get a loan unless it is very small or dep on other factors.
    
    Within grades they also define subgrades that this program does not implement:
    A1 A2 A3 B1 B2 B3 C1 C2 C3 D1 D2 D3 E1 E2 E3
    '''
    
    A,B,C,D,E = 0,1,2,3,4
    
    for i in range(0,parts):
        for j in range(ceil(i * lines_for_each_part),  ceil((i+1) * lines_for_each_part)):
            
            per = {}

            if i == A:
                if randint(0,18) == 1:
                    # Maybe some random people dont get a loan in A
                    per['accepted'] = 0
                else:
                    per['accepted'] = 1
                per['health_ins'] = randint(0,1)
                per['creditscore'] = randint(353,400)
                per['creditgrade'] = 'A'
            elif i == B:
                if randint(0,7) == 1:
                    # Maybe some random people dont get a loan in B
                    per['accepted'] = 0
                    per['health_ins'] = randint(0,1)
                    
                else:
                    per['accepted'] = 1
                    per['health_ins'] = 1
                
                per['creditscore'] = randint(317,353)
                per['creditgrade'] = 'B'

            elif i == C: 
                n = randint(0,1)
                per['accepted'] = n
                per['health_ins'] = n
                per['creditscore'] = randint(281,317)
                
                per['creditgrade'] = 'C'

            elif i == D:
                if randint(0,3) == 1:
                    per['accepted'] = 1
                    per['health_ins'] = 1
                else:
                    per['accepted'] = 0
                    per['health_ins'] = 0
                
                per['creditscore'] = randint(235,281)
                per['creditgrade'] = 'D'

            elif i == E:
                if randint(0,40) == 1:
                    # Maybe some random people in E get a loan
                    per['accepted'] = 1
                    per['health_ins'] = 1
                else:
                    per['accepted'] = 0
                    per['health_ins'] = 0
                per['creditscore'] = randint(100,235)

                per['creditgrade'] = 'E'
            
            ppl.append(per)


def gen_age(per):
    if per['accepted'] == 1:
        return randint(18, 90)
    elif per['creditgrade'] == 'E':
        return randint(55,90)
    elif per['creditgrade'] == 'D':
        if randint(0,4) == 2:
            return randint(18,30)
        return randint(55,90)
    else:
        return randint(18, 90)

def gen_marital(per):
    if per['age'] > 78:
        return 0
    if per['accepted'] == 1:
        if randint(0,6) == 3:
            return 0
        else:
            return 1
    if per['accepted'] == 0:
        if randint(0,2) == 1:
            return 1
        else:
            return 0

def gen_cscore(per):
    '''We will be basing the credit score the risk factor 
    The Credit Info Risk Factor Grade that are defined as follows
    A = 353-400+ points, B 317-353 points, C = 281-317, D = 235-281, E = 100-235'''
    if['creditgrade'] == 'A':
        return randint(353,400)
    if['creditgrade'] == 'B':
        return randint(317,352)
    if['creditgrade'] == 'C':
        return randint(281,316)
    if['creditgrade'] == 'D':
        return randint(235,280)
    if['creditgrade'] == 'E':
        return randint(100,235)
def gen_amount(per):
    if per['accepted'] == 1:
        if per['age'] > 50 or per['age'] < 25:
            # Small loan half mil to 10 mil
            return randint(500000,15000000)
        if per['creditgrade'] == 'E':
            # Small loan
            return randint(100000,6000000)
        else:
            # Possibly bigger loan
            return randint(1000000,70000000)
    else:
        return randint(1000000,100000000)    

def add_to_file(ppl):
    with open(loan_info_file, 'w') as f:
        f.write('accepted,creditscore,amount,age,marital,health_ins,creditgrade\n')
        # Here you can sort the file by other things if you want
        for per in sorted(ppl, key=lambda x: x['accepted']):
            f.write('%s,%s,%s,%s,%s,%s,%s\n' % (per['accepted'], per['creditscore'], per['amount'],per['age'],per['marital'],per['health_ins'],per['creditgrade']))

def main():
    # This list will store a dict for each line
    ppl = []
    
    if is_not_zero_file(loan_info_file):
        print("reading old file...")
        read_file(ppl)
    
    if not ppl:
        create_new(ppl)
        
    for per in ppl:
        '''This is the order in which we will determine each factor
        create_new() handles accepted, creditgrade, health_ins and creditscore'''

        if 'age' not in per:
            per['age'] = gen_age(per)
        if 'marital' not in per:
            per['marital'] = gen_marital(per)
        if 'creditscore' not in per:
            per['creditscore'] = gen_cscore()
        if 'amount' not in per:
            per['amount'] = gen_amount(per)
    
    add_to_file(ppl)
    print("Length of new file", loan_info_file, "is:" , len(ppl))
    print("Last created at" ,str(datetime.now()))
